In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup as bs
import re
from tqdm.notebook import trange, tqdm
from tqdm.gui import tqdm as tqdm_gui
from IPython.display import display
import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt

импорт библиотек

In [2]:
import warnings
warnings.filterwarnings('ignore')

отключение ворнингов

In [3]:
pd.set_option('display.max_colwidth', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)

задание параметров таблиц

In [4]:
def find_id(x):
    xxx = x.find('span', class_="ty-control-group__item").text  
    return xxx

функция создания столбца с кодом товара

In [5]:
def find_availability(t):
    xxx = (t.find('a', class_="ty-control-group product-list-field cm-external-click")
           .text.replace('\ue924','').replace('\ue86c','').replace('\n',''))
    return xxx  

функция создания столбца с парсингом наличия товара

In [6]:
def find_title(t):
    xxx = t.find('a', class_="product-title").text
    return xxx

функция создающая столбец с названием товара

In [7]:
def find_price(t):
    xxx = t.find('span', class_="ty-price-num").text.replace(u'\xa0',u'')
    return xxx

функция создающая столбец с продажной ценой товара

In [8]:
def find_old_price(t):
    try:
        xxx = t.find('span', class_="ty-strike").text.replace(u'\xa0',u'').replace(u'руб',u'')
        return xxx
    except:
        return np.NaN

функция созжающая столбец с предидущей ценой для акционного товара

In [9]:
def find_url(t):
    xxx = t.find('a', class_="abt-single-image")['href']
    return xxx

Функция создающая столбец с урл адресом товара

In [10]:
def preprocessing(x):
    print(f'таблица имеет следующий вид:')
    display(x.head())
    print('*'*100)
    print(f'таблица имеет {x.shape[0]} сторок и {x.shape[1]} столбцов')
    print('*'*100)
    print(f'в таблице обнаружено дублекатов:{x.duplicated().sum()}')
    print('*'*100)
    print('в таблице обнаружены пропуски в следующих столбцах:')
    display(x.isna().sum())
    print('*'*100)
    print('Эти пропуски составлябт следующее количество в процентах')
    display((x.isna().mean()* 100).round(1))
    print('*'*100)
    print('столбцы имеют следующие типы:')
    display(x.dtypes)
    x.columns = [y.lower().replace(' ', '_') for y in x.columns.values] 
    print('*'*100)
    print('названия столбцов переписаны в нижнем регистре, пробелы заменены на нижнее подчеркивание')
    print('*'*100)
    display(x.info())

Функция для проверки датафрейма(вид дублекаты пропуски)

In [11]:
def parsing_2(url):
    data = pd.DataFrame(columns=['url', 'id', 'date', 'availability', 'title', 'price', 'old_price'])
    for i in tqdm(url):        
        q = 1
        while q>0:        
            reqx = requests.get(i + str(q))
            soup = bs(reqx.text, 'html.parser')
            datax = pd.DataFrame({'url':pd.Series(soup.findAll('div', class_='ypi-grid-list__item_body'),dtype = 'object')})
            data = pd.concat([data, datax], ignore_index=True)
            if len(list(soup.findAll('div', class_='ypi-grid-list__item_body'))) > 0:
                q = q + 1
            else:
                q = 0
    data['id'] = data['url'].apply(find_id)
    data['date'] = pd.to_datetime('today').normalize()
    data['availability'] = data['url'].apply(find_availability)
    data['title'] = data['url'].apply(find_title)
    data['price'] = data['url'].apply(find_price)
    data['old_price'] = data['url'].apply(find_old_price)
    data['url'] = data['url'].apply(find_url)
    data['price'] = data['price'].astype(float)
    data.loc[data['old_price'] == '','old_price'] = np.nan
    data['old_price'] = data['old_price'].astype(float)
    return data

Функция предварительного парсинга сайта, которая проходит по всем страницам в урл адресах переменной `all_url` и создает датафрейм в котором храняться: урл адреса товаров, индентефикатор товаров, дата предварительного парсинга товара, наличия товара, названия товара, цены товара, старой цены для акционного товара.

# Парсинг цен кодов товара и url

In [12]:
all_url = ['https://hi-tech.md/televizory-i-elektronika/page-',
           'https://hi-tech.md/bytovaya-tehnika/page-',
           'https://hi-tech.md/kompyuternaya-tehnika/page-',
           'https://hi-tech.md/mebel-tekstil/kompyuternaya-mebel/ofisnye-kresla-i-stulya/komp.-kresla/page-',
           'https://hi-tech.md/instrumenty-i-oborudovanie/elektroinstrument/page-',
           'https://hi-tech.md/tovary-dlya-doma/page-'
          ]

all_url = ['https://hi-tech.md/televizory-i-elektronika/page-']

all_url = ['https://hi-tech.md/bytovaya-tehnika/page-']

all_url = ['https://hi-tech.md/kompyuternaya-tehnika/page-']

all_url = ['https://hi-tech.md/mebel-tekstil/kompyuternaya-mebel/ofisnye-kresla-i-stulya/komp.-kresla/page-']

all_url = ['https://hi-tech.md/instrumenty-i-oborudovanie/elektroinstrument/page-']

all_url = ['https://hi-tech.md/tovary-dlya-doma/page-']

переменная со списком ссылок на сайт для парсинга

In [ ]:
df = parsing_2(all_url)
df

  0%|          | 0/6 [00:00<?, ?it/s]

предварительный парсинг

In [ ]:
df = df.drop_duplicates().reset_index(drop=True)

удаление дублекатов

In [ ]:
df.to_csv('csv/hi_tech_test_'+dt.datetime.now().strftime('%Y_%m_%d')+'.csv', index=False)

создание резервной копии парсинга данных

In [ ]:
df = pd.read_csv('csv/hi_tech_test_'+dt.datetime.now().strftime('%Y_%m_%d')+'.csv')

In [ ]:
preprocessing(df)

In [ ]:
hai_group = pd.read_csv('csv/ahay_group.csv')

In [ ]:
df = df.merge(hai_group, how='left', on='id')

In [ ]:
df_new = df[(df['_category'].isna()) | (df['_subcategory'].isna()) | (df['_group'].isna()) | (df['_subgroup'].isna())]

In [ ]:
df = df[~((df['_category'].isna()) | (df['_subcategory'].isna()) | (df['_group'].isna()) | (df['_subgroup'].isna()))]

загрузка резервной копии парсинга данных

df = df.sample(30)

тестовое уменьшение таблици

In [ ]:
def xxx(x):
    reqx = requests.get(x)
    soup = bs(reqx.text, 'html.parser')    
    elem_2 = soup.findAll('div', class_='ty-breadcrumbs clearfix')
    name = soup.findAll('span', class_='ty-breadcrumbs__current')
            
    try:
        names = name[0].text
        name_group = elem_2[0].text.replace('/'+str(names),'').replace('\n','')
        err = 'err'
    except:
        names = 'Noname'
        name_group = 'Noname/Noname/Noname/Noname/Noname/Noname'
        err = 'Noname_err'
    try:
        len_name = len((elem_2[0].text.replace('/'+str(names),'').replace('\n','')).split('/'))
    except:
        len_name = 1
        
    try:
        if len_name <= 1:
            d_14 = 'err1'
            d_15 = 'err1'
            d_16 = 'err1'
            d_17 = 'err1'
        elif len_name <= 2:
            d_14 = 'H_'+(name_group.split("/")[1]).replace(',','').replace(' ','_').replace('-','_').lower()
            d_15 = 'H_'+(name_group.split("/")[1]).replace(',','').replace(' ','_').replace('-','_').lower()
            d_16 = 'H_'+(name_group.split("/")[1]).replace(',','').replace(' ','_').replace('-','_').lower()
            d_17 = 'H_'+(name_group.split("/")[1]).replace(',','').replace(' ','_').replace('-','_').lower()      
        elif len_name <= 3:
            d_14 = 'H_'+(name_group.split("/")[1]).replace(',','').replace(' ','_').replace('-','_').lower()
            d_15 = 'H_'+(name_group.split("/")[2]).replace(',','').replace(' ','_').replace('-','_').lower()
            d_16 = 'H_'+(name_group.split("/")[2]).replace(',','').replace(' ','_').replace('-','_').lower()
            d_17 = 'H_'+(name_group.split("/")[2]).replace(',','').replace(' ','_').replace('-','_').lower()
        elif len_name == 4:
            d_14 = 'H_'+(name_group.split("/")[1]).replace(',','').replace(' ','_').replace('-','_').lower()
            d_15 = 'H_'+(name_group.split("/")[2]).replace(',','').replace(' ','_').replace('-','_').lower()
            d_16 = 'H_'+(name_group.split("/")[3]).replace(',','').replace(' ','_').replace('-','_').lower() 
            d_17 = 'H_'+(name_group.split("/")[3]).replace(',','').replace(' ','_').replace('-','_').lower()
        elif len_name > 4:
            d_14 = 'H_'+(name_group.split("/")[1]).replace(',','').replace(' ','_').replace('-','_').lower()
            d_15 = 'H_'+(name_group.split("/")[2]).replace(',','').replace(' ','_').replace('-','_').lower()
            d_16 = 'H_'+(name_group.split("/")[3]).replace(',','').replace(' ','_').replace('-','_').lower() 
            d_17 = 'H_'+s(name_group.split("/")[4]).replace(',','').replace(' ','_').replace('-','_').lower()
    except:
        d_14 = 'err0'
        d_15 = 'err0'
        d_16 = 'err0'
        d_17 = 'err0'
        
    return pd.Series([(d_14), (d_15), (d_16), (d_17)])

Функция основного парсинга, которая для каждой строки по урл адресу парсит и находит наличие товара в магахинах а также группу товара, категорию товара, подкатегорию товата и количество подкатегорий товара

# Парсинг групп товара

In [ ]:
tqdm.pandas() 
df_new[['_category','_subcategory','_group','_subgroup']] = df_new['url'].progress_apply(xxx)

In [ ]:
data = df_new[~((df_new['_category']=='err1') | (df_new['_category']=='err0'))].reset_index(drop=True)
data = data[['id', '_category','_subcategory','_group', '_subgroup']]
data = data.sort_values(['_category','_subcategory', '_group', '_subgroup'])

In [ ]:
hai_group = pd.concat([hai_group, data], ignore_index=True).sort_values(['_category','_subcategory', '_group', '_subgroup']).reset_index(drop=True)
hai_group.to_csv('csv/ahay_group.csv', index=False)  

In [ ]:
df = pd.concat([df, df_new], ignore_index=True).sort_values(['_category','_subcategory', '_group', '_subgroup']).reset_index(drop=True)

Основной парсинг

In [ ]:
df._category.unique()

In [ ]:
df._subcategory.unique()

In [ ]:
df._group.unique()

In [ ]:
df[df['_category']=='err1'].id.count()

In [ ]:
df[df['_category']=='err0'].id.count()

In [ ]:
df[df['_category']=='err1']

In [ ]:
df[df['_category']=='err0']

In [ ]:
preprocessing(df)

In [ ]:
df.to_csv('csv/hi_tech_test_'+dt.datetime.now().strftime('%Y_%m_%d')+'.csv', index=False)

создание резервной копии парсинга данных

In [ ]:
df = pd.read_csv('csv/hi_tech_test_'+dt.datetime.now().strftime('%Y_%m_%d')+'.csv')

загрузка резервной копии парсинга данных

In [ ]:
df.head(2)

In [ ]:
df['error_group'] = 0
df.loc[df['_subgroup'].isna(), 'error_group'] = 1
df.loc[df['_subgroup']=='err0', 'error_group'] = 1
df.loc[df['_subgroup']=='err1', 'error_group'] = 1

Создание столбцов с флагом ошибки для товаров, для которых нет данных по остаткам хотябы в одном магазине

In [ ]:
df.loc[df['error_group']==1 , ['_category',
       '_subcategory', '_group', '_subgroup']] = 'H_no_group'

удаление наличия товара в магазинах и наименовании группы если у товара есть ошибки в этих данны

Изменение типов данных столбцов наличия товара в определенном магазине на тип данных `float`  и создания столбцов по наличию товара в магазинах городов

In [ ]:
df = df[['url', 'id', '_category', '_subcategory', '_group', '_subgroup', 'title',
               'date',   'price', 'old_price', 'availability', 'error_group', ]]

изменение порядка располажения столбцов

In [ ]:
preprocessing(df)

просмотр данных после парсинга

# Группировка товара

- разобраться с креслами в ногруп

In [ ]:
df.loc[df['_category']=='H_товар_без_акции', '_category'] = 'H_no_group'
df.loc[df['_category']=='H_товар_по_акции', '_category'] = 'H_no_group'
df.loc[df['_subcategory']=='H_мультимедиа', '_subcategory'] = 'H_аудиосистемы'
df.loc[df['_group']=='H_акустические_системы', '_group'] = 'H_акустика'
df.loc[df['_group']=='H_музыкальная_техника', '_group'] = 'H_акустика'
df.loc[df['_subcategory']=='H_умный_дом_с_яндекс','_subcategory'] = 'H_аудиосистемы'
df.loc[df['_group']=='H_яндекс_станции', '_group'] = 'H_акустика'
df.loc[df['_group']=='H_airpods', '_subcategory'] = 'H_аудиосистемы'
df.loc[df['_group']=='H_airpods', '_group'] = 'H_наушники'
df.loc[df['_group']=='H_apple_watch', '_group'] = 'H_часы_и_браслеты'
df.loc[df['_group']=='H_iphone', '_group'] = 'H_смартфоны'
df.loc[df['_group']=='H_ipad', '_group'] = 'H_планшеты'
df.loc[df['_group']=='H_macbook', '_group'] = 'H_ноутбуки'
df.loc[df['_group']=='H_imac', '_group'] = 'H_моноблоки'
df.loc[df['_group']=='H_часы_и_браслеты', '_subcategory'] = 'H_гаджеты'
df.loc[df['_group']=='H_смартфоны', '_subcategory'] = 'H_смартфоны_и_аксессуары'
df.loc[df['_group']=='H_планшеты', '_subcategory'] = 'H_готовые_решения'
df.loc[df['_group']=='H_ноутбуки', '_subcategory'] = 'H_готовые_решения'
df.loc[df['_group']=='H_моноблоки', '_subcategory'] = 'H_готовые_решения'
df.loc[df['_subcategory']=='H_стиральные_и_сушильные_машины', '_group'] = 'H_стиральные_машины'
df.loc[df['_subgroup']=='H_сушильные_машины', '_group'] = 'H_сушильные_машины'
df.loc[df['_subgroup']=='H_наклейки', '_group'] = 'H_наклейки'
df.loc[df['_subgroup']=='H_подставки', '_group'] = 'H_подставки'
df.loc[df['_subgroup']=='H_замки', '_group'] = 'H_замки'
df.loc[df['_subgroup']=='H_usb_hub', '_group'] = 'H_usb_hub'
df.loc[df['_subgroup']=='H_сумки_и_чехлы', '_group'] = 'H_сумки_и_чехлы'
df.loc[df['_subgroup']=='H_защитные_стекла', '_group'] = 'H_защитные_стекла'
df.loc[df['_subcategory']=='H_мониторы','_group'] = 'H_мониторы'

In [ ]:
for i in df['_category'].unique():
    t = df[df['_category']==i]
    print(i)
    print(t['_subcategory'].unique())
    print('-'*100)

In [ ]:
for i in df['_subcategory'].unique():
    t = df[df['_subcategory']==i]
    print(i)
    print(t['_group'].unique())
    print('-'*100)

просмотр групп товаров в подкатегориях

In [ ]:
for i in df['_group'].unique():
    t = df[df['_group']==i]
    print(i)
    print(t['_subgroup'].unique())
    print('-'*100)

In [ ]:
df._category.unique()

In [ ]:
df._subcategory.unique()

In [ ]:
df._group.unique()

In [ ]:
df._subgroup.unique()

# Подготовка анализа

In [ ]:
df.head(2)

In [ ]:
brend = pd.read_excel('csv/brend.xlsx')

In [ ]:
def lower(x):
    return x.lower()
brend['names'] = brend['name'].apply(lower)
brend_up = brend.name.unique()
brend_low = brend.names.unique()

Создаем список брендов

In [ ]:
def brends(x):
    xx = x.replace('\xa0', ' ')
    xx = xx.split(' ')    
    for i in range(len(xx)-1):
        if xx[i].lower() in brend_low:
            return xx[i].lower()
        elif (str(xx[i].lower())+' '+str(xx[i+1].lower())) in brend_low:
            return (str(xx[i].lower())+' '+str(xx[i+1].lower()))    
    else:    
        return 'другой'
tqdm.pandas()
df['brend'] = df['title'].progress_apply(brends)

создаем столбец с брендами

In [ ]:
df['action_sale'] = 0
df.loc[df['old_price'] > 0,'action_sale'] = 1

Создаем столбец с флагом акции

In [ ]:
df['sale'] = (100 - ((df['price']*100)/df['old_price'])).round(2) 

создаем столбец показывающий скидку в процентах

In [ ]:
def count_group(x):
    return df[df['_group']==x]['id'].count()    
tqdm.pandas()
df['count_group'] = df['_group'].progress_apply(count_group)

создание столбца с количеством товара в группе

In [ ]:
def count_group_presence(x):
    return df[(df['availability']=='В наличии')&(df['_group']==x)]['id'].count()   
tqdm.pandas()
df['activ_count_group'] = df['_group'].progress_apply(count_group_presence)

создание столбца с количеством товара, который есть в наличии, в группе

In [ ]:
def dif_price(x):    
    max_price = df[(df['availability']=='В наличии')&(df['_group']==x)]['price'].max()
    min_price = df[(df['availability']=='В наличии')&(df['_group']==x)]['price'].min() 
    return max_price / min_price
tqdm.pandas()
df['dif_price'] = df['_group'].progress_apply(dif_price)

создам столбец с отношением самой дорогой цены в группе к самой дешевой.

In [ ]:
df['group_prise'] = df.groupby('_group')['price'].transform(lambda x: pd.cut(x, bins = 4, labels=["бюджет", "стандарт", 'дорогой', "элитный"]))

создание столбца с ценовой категорией товара

In [ ]:
df = df[['url', 'id', '_category', '_subcategory', '_group', '_subgroup', 'brend', 
         'title', 'date', 'price', 'old_price', 'action_sale', 'sale', 'group_prise', 'dif_price',
         'availability', 'count_group', 'activ_count_group','error_group']]

In [ ]:
df.drop('_subgroup', axis= 1 , inplace= True )

In [ ]:
df.to_csv('csv/hi_tech_final_'+dt.datetime.now().strftime('%Y_%m_%d')+'.csv', index=False) 

# Аналитика акционных товаров

In [ ]:
df.head(3)

In [ ]:
df.groupby('_group')['price'].mean().reset_index()

In [ ]:
plt.rcParams.update(plt.rcParamsDefault)

In [ ]:
df[df['_group']=='H_пылесосы'].head(2)

In [ ]:
separator = ['_group','brend','group_prise']

In [ ]:
for x in separator:
    for i in df['_subcategory'].unique():        
        plt.figure(figsize=(16,9))
        sns.barplot(df[df['_subcategory']==i].groupby([x ,'action_sale'])['id'].count().sort_values(ascending=False).reset_index(),
                       x=x, y='id', hue='action_sale')
        plt.xticks(rotation=20)
        plt.title(f'количество товаров разных {x} в подкатегории {i} распределенные по наличию акции ')
        plt.xlabel(f' {x}')
        plt.ylabel('количество товара')
        plt.show()

In [ ]:
df[df['_subcategory']=='пылесосы_и_аксессуары'].groupby(['_group' ])['action_sale'].mean().reset_index()

In [ ]:
for x in separator:
    for i in df['_subcategory'].unique():        
        plt.figure(figsize=(16,9))
        sns.barplot(df[df['_subcategory']==i].groupby(x)['action_sale'].mean().sort_values(ascending=False).reset_index(),
                       x=x, y='action_sale', color='blue')
        plt.xticks(rotation=20)
        plt.title(f'количество товаров разных {x} в подкатегории {i} ')
        plt.xlabel(f' {x}')
        plt.ylabel('доля акционного товара')
        plt.show()